<a href="https://colab.research.google.com/github/joeyk417/echo-synergy/blob/main/echo_synergy_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing the dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio
!pip install langchain
!pip install pypdf
!pip install dotenv
!pip install sentence_transformers
!pip install chromadb
!pip install replicate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255313 sha256=4c7b71a155589d9fe94d019002a122531be04212a1b4dc44ce39762563934edc
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

import os
import random
from typing import List, Dict
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import Replicate
from langchain.prompts import PromptTemplate
import replicate

In [ ]:
###############Prompt##########################3
prompt_template_questions = """
Can you play a role as a customer at Pharmacy.
Generate a list of 30 thoughtful questions and list them as bullet-point.
Those questions are customer at Pharmacy will ask.
The questions should be open-ended, encourage deep thinking.
It should be about medications, understanding prescription requirements,
expressing gratitude,
and concluding the interaction. Avoid questions that can be answered with a simple yes or no

------------
{text}
------------

Create questions that will you have as an customer in a pharmacy. Make sure not to lose any important information.
Can you play a role as a customer at Pharmacy. And I am a pharmacist. Can you ask me one short question as a customer?
You will ask me 10 potenial questions. So we will talk as real conversations?

QUESTIONS:
"""

PROMPT_QUESTIONS_LIST = PromptTemplate(template=prompt_template_questions,
                                  input_variables=["text"])

refine_template_questions = """
You are an customer in a pharmacy. You goal is performing specific tasks: greeting the pharmacist, inquiring
about medications, understanding prescription requirements, expressing gratitude, and concluding the
interaction.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below. Can you play a role as a customer at Pharmacy. And I am a pharmacist. Can you ask me some questions as a customer?
You ask me 10 potenial questions. So we will talk as real conversations?
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.

QUESTIONS:
"""

REFINE_PROMPT_QUESTIONS = PromptTemplate(
  input_variables=["existing_answer", "text"],
  template=refine_template_questions,
)

# # This controls how the standalone question is generated.
# # Should take `chat_history` and `question` as input variables.
# ConversationTemplate = (
#     # "Combine the chat history and follow up question into "
#     # "a standalone question. Chat History: {chat_history}"
#     # "Follow up question: {question}"
#     "Can you play a role as a customer at Pharmacy. And I am a pharmacist. Can you ask me 1 short question as a customer based on our chat history"llm_answer_gen
#     ------------
#     {text}
# ------------
# )

ConversationTemplate = """
"Can you play a role as a customer at Pharmacy. And I am a pharmacist. Can you ask me 1 short question as a customer based on our chat history"
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.

QUESTIONS:
"""
CONVERSATION_PROMPT_QUESTIONS = PromptTemplate(template=ConversationTemplate,
                                               input_variables=["text"])


In [ ]:
##################################################
# Load documents
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files ##Colab Production code
uploaded = files.upload()

Mounted at /content/drive


Saving Basicsofpharmacy.pdf to Basicsofpharmacy.pdf


In [ ]:
# pdf_path = os.path.join("data", "Basicsofpharmacy.pdf")
loader = PyPDFLoader('/content/Basicsofpharmacy.pdf') #Colab Production code
data = loader.load()

## To do, load docs from .data folder
# reader = SimpleDirectoryReader("./data")
# documents = reader.load_data()
# print("documents loaded successfully.")

In [ ]:
# Combine text from Document into one string for question generation
text_question_gen = ""
for page in data:
  text_question_gen += page.page_content

# Initialize Text Splitter for question generation
text_splitter_question_gen = RecursiveCharacterTextSplitter(chunk_size=10000,
                                                            chunk_overlap=50)

# Split text into chunks for question generation
text_chunks_question_gen = text_splitter_question_gen.split_text(
  text_question_gen)

# Convert chunks into Documents for question generation
docs_question_gen = [
  Document(page_content=t) for t in text_chunks_question_gen
]

In [ ]:
#####Chains################################

# get a token: https://replicate.com/account ##Colab Production code
# from getpass import getpass
# import os

from google.colab import userdata #Colab codes
from pyngrok import conf

os.environ["REPLICATE_API_TOKEN"] = userdata.get("REPLICATE_API_TOKEN")
replicate_api_token = os.environ["REPLICATE_API_TOKEN"]

# REPLICATE_API_TOKEN = getpass()
# os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN
# Initialize Large Language Model for question generation
# replicate_api_token = os.getenv('r8_DBCpxyYhxOEbWCqf61d9SbYvp5YxSb51RJuvI')
llm_question_gen = Replicate(
  model=
  "meta/llama-2-7b-chat:f1d50bb24186c52daae319ca8366e53debdaa9e0ae7ff976e918df752732ccc4",
  is_chat_model=True,
  system_prompt = "You are a customer at Phamacy",
  input={
    "temperature": 0.01,
    "max_length": 900,
    "top_p": 1
  },
)

question_gen_chain = load_summarize_chain(
  llm=llm_question_gen,
  chain_type="refine",
  verbose=True,
  question_prompt=PROMPT_QUESTIONS_LIST,
  refine_prompt=REFINE_PROMPT_QUESTIONS,
)

question_new_gen_chain = load_summarize_chain(
  llm=llm_question_gen,
  chain_type="refine",
  verbose=True,
  question_prompt=CONVERSATION_PROMPT_QUESTIONS,
  refine_prompt=REFINE_PROMPT_QUESTIONS,
)


def generate_questions(conversation_history):
  # Placeholder for the actual logic to generate questions
  question_gen_chain.run(docs_question_gen)


def generate_new_questions(conversation_history):
  question_new_gen_chain.run(docs_question_gen)


                    Please confirm that is_chat_model is what you intended.
                    Please confirm that system_prompt is what you intended.


In [ ]:
### Save Questions in File ###################
from google.colab import files
uploaded = files.upload()
file_path = '/content/questions.txt'


Saving questions.txt to questions.txt


In [ ]:
with open(file_path, 'r') as file_obj:
  # read first character
  first_char = file_obj.read(1)

  if not first_char:
    questions = question_gen_chain.run(docs_question_gen)
  else:
    questions = ""

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Can you play a role as a customer at Pharmacy.
Generate a list of 30 thoughtful questions and list them as bullet-point.
Those questions are customer at Pharmacy will ask.
The questions should be open-ended, encourage deep thinking.
It should be about medications, understanding prescription requirements,
expressing gratitude,
and concluding the interaction. Avoid questions that can be answered with a simple yes or no

------------
OD1N Pharmaceuticals AI Training Document  
Section 1: Introduction to Pharmaceutical Terminology  
• Antibiotics:  Medications that combat bacterial infections. Common examples:  
o Amoxicillin:  Used for strep throat, pneumonia, skin infections, and urinary 
tract infections. Typically taken orally.  
o Azithromycin:  Treats various infections including bronchitis, pneumonia, and 
infections of the ears, lungs, sinuses, skin, throat, and reproductive 

In [ ]:
# from sentence_transformers import SentenceTransformer

# # Initialize the model
# model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# # Your texts to encode
# texts = ["Example sentence 1", "Example sentence 2"]

# Create vector database for answer generation

import chromadb
import chromadb.config
embeddings = HuggingFaceEmbeddings(
  model_name="sentence-transformers/all-MiniLM-L6-v2",
  model_kwargs={"device": "cpu"})

# # Initialize vector store for answer generation
vector_store = Chroma.from_documents(docs_question_gen, embeddings)

# Split generated questions into a list of questions
question_list = questions.split("\n")
print(question_list)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[" Of course! I'd be happy to play the role of a customer at a pharmacy and ask you questions as a pharmacist. Here are 10 potential questions I might ask:", '1. Can you explain the difference between over-the-counter (OTC) medications and prescription medications?', '2. How do I properly dispose of unused medications to prevent accidental ingestion or misuse?', "3. I'm taking several medications for different conditions. Can you help me ensure that I'm taking them correctly and safely?", "4. I'm experiencing side effects from a medication I've been taking for a few days. What should I do?", "5. Can you recommend a medication for my chronic pain that won't cause addiction or dependence?", "6. I'm concerned about the cost of my medications. Are there any options available to help me save money?", '7. Can you explain how my medications interact with my other health conditions, such as diabetes or high blood pressure?', "8. I'm taking a medication that I'm not sure I need anymore. Can you

In [ ]:

questions_string = '\n'.join(question_list)

with open("/content/questions.txt", 'w') as writefile:
    writefile.write(questions_string)


In [ ]:
print("---------------------------------------------------------\n\n")
print("-------------------Completed RAG Process-----------------\n\n")


##############################################

---------------------------------------------------------


-------------------Completed RAG Process-----------------




In [ ]:
app = FastAPI()
@app.get("/")
def root():
  return {"hello world"}

In [ ]:
# Endpoint to receive the first question and start the conversation
@app.get("/start/")
async def start_conversation():
  # Generate the first question
  greetings = ["Hello", "How are you doing", "Hi there","Howdy"," How's it going","What's up",
               "What's going on","How's your day going","How's your day"]

  random_greeting = random.choice(greetings)

  # Return the first question to the user
  return {"greeting": random_greeting}

In [ ]:
@app.get("/first-question/")
async def get_question():
  with open('/content/questions.txt') as f:
    questions = f.readlines()
    print(questions)
    random_question = random.choice(questions)
  print(random_question)
  return {"first-question": random_question}

In [ ]:
# Endpoint to receive and process the user's answer
@app.post("/answer/")
async def process_answer(answer: str):
  # Ensure conversation_history is properly initialized as a list
  global conversation_history
  if not isinstance(conversation_history, list):
    conversation_history = []

  # Append the user's answer to the conversation history
  conversation_history.append(answer)

  # Generate a new question based on the conversation history
  new_question = generate_questions(conversation_history)

  # Return the new question to the user
  return {"question": new_question}

In [ ]:
# Endpoint to receive the next question based on the previous answer
@app.post("/next/")
async def next_question(answer: str):
  conversation_history.append(answer)
  # Generate the next question based on the conversation history
  next_question = question_new_gen_chain.run(docs_question_gen)
  print(next_question)
  questions_list = next_question.split("\n")  # Split by newline character
  print(questions_list)
  # Filter out empty strings and non-question lines
  questions_list = [
    line.strip() for line in questions_list
    if line.strip() and line.strip().endswith("?")
  ]
  random_new_question = random.choice(questions_list)
  print(random_new_question)
  return {"question": random_new_question}


In [ ]:
from google.colab import userdata

os.environ["NGROK"] = userdata.get("NGROK")
conf.get_default().auth_token = os.environ["NGROK"]

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [2452]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://cf30-34-135-203-19.ngrok-free.app
INFO:     203.220.168.125:0 - "GET / HTTP/1.1" 200 OK
INFO:     203.220.168.125:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     203.220.168.125:0 - "GET / HTTP/1.1" 200 OK
INFO:     203.220.168.125:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     203.220.168.125:0 - "GET /openapi.json HTTP/1.1" 200 OK
[" Of course! I'd be happy to play the role of a customer at a pharmacy and ask you questions as a pharmacist. Here are 10 potential questions I might ask:\n", '1. Can you explain the difference between over-the-counter (OTC) medications and prescription medications?\n', '2. How do I properly dispose of unused medications to prevent accidental ingestion or misuse?\n', "3. I'm taking several medications for different conditions. Can you help me ensure that I'm taking them correctly and safely?\n", "4. I'm experiencing side effects from a medication I've been taking for a few days. What should I do?\n", "5. Can you recommend a medicatio

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 411, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 69, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 164, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dis

INFO:     2a09:bac2:3d9:1d7::2f:91:0 - "POST /answer/?answer=One%20requires%20a%20prescription%20the%20other%20does%20not HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 411, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 69, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 164, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dis